In [1]:
# install it if you did not yet!

#!pip install simple-network

In [ ]:
# import library :

from simpleN import MultilayerNetwork, Visualize

In [2]:
# Create an Object of it :

g = MultilayerNetwork()

In [3]:
# adding layers !

g.add_layer('one')
g.add_layer('two')

In [4]:
# adding nodes to layer one !

for i in range(50):
    g.add_node(layer_name='one', node = i)

In [5]:
# adding nodes to layer two !

for i in range(50, 100):
    g.add_node(layer_name='two', node = i)

In [6]:
# Add Between layer edges !

g.add_edge(node1=5, node2= 80 , layer_name1= 'one', layer_name2='two')
g.add_edge(node1=17, node2= 55 , layer_name1= 'one', layer_name2='two')
g.add_edge(node1=6, node2= 90 , layer_name1= 'one', layer_name2='two')
g.add_edge(node1=47, node2= 52 , layer_name1= 'one', layer_name2='two')
g.add_edge(node1=5, node2= 60 , layer_name1= 'one', layer_name2='two')
g.add_edge(node1=15, node2= 99 , layer_name1= 'one', layer_name2='two')

In [7]:
# Add Inside layer edges !

g.add_edge(node1=6, node2= 27 , layer_name1= 'one', layer_name2='one')
g.add_edge(node1=22, node2= 47 , layer_name1= 'one', layer_name2='one')
g.add_edge(node1=14, node2= 15 , layer_name1= 'one', layer_name2='one')

g.add_edge(node1=71, node2= 90 , layer_name1= 'two', layer_name2='two')
g.add_edge(node1=66, node2= 99 , layer_name1= 'two', layer_name2='two')

In [8]:
# adj_matrix = g.edges

In [12]:
Visualize(g).show_graph()